In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

# 前置准备

## 查看Pytorch版本

In [4]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.7
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.8, CUDNN_VERSION=8.7.0, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUS

## 定义全局变量

In [5]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
# train_data_path = 'data/zhouyi_dataset_handmade.csv'    # 训练数据路径
train_data_path = 'data/xxx.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 16                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

# 数据处理

In [6]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path, encoding="utf-8")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 4
    })
})


In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"], num_examples=4)

,content,summary
0,智慧交通观察报告专栏是哪些机构开设的？,中国交通报社联合腾讯开设《智慧交通观察报告·2022》专栏，坚持贯彻《交通强国建设纲要》《国家综合立体交通网规划纲要》等重大战略、重要规划、重点政策要求，助力推动交通运输行业智慧转型，同时，进一步探讨新技术与交通行业深度融合的典型场景和实现路径。
1,智慧交通观察报告-2022是谁发布的？,市场圆桌对话，六次行业调研，而是为行业专家建言献策，最终基于10万字手稿汇编整理。
2,为什么说我国高速公路基础设施建设走到了智慧化的关口？,随着车辆、设施网络的形成，交通事故、拥堵、排放等问题也随之而来，要解决交通发展带来的问题，既需要运载工具智能化，也需要以数字化和智能化的方式改造基础设施。
3,智慧交通观察报告的序言主题是什么？,加快建设交通强国 智慧交通发展迎来全新机遇。


In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

In [9]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}

In [10]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

In [11]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()

In [12]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}

In [13]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

# 加载模型

In [18]:
!export HF_DATASETS_CACHE="/root/autodl-tmp/cache/huggingface/hub"


from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True,
                                  revision='b098244', 
                                  cache_dir="/root/autodl-tmp/cache/huggingface/hub")

model.supports_gradient_checkpointing = True  
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [19]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [20]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [22]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()
qlora_model.eval()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=4096, out_features=4608, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=4608, bias=False)

# 模型微调(QLoRA)

In [23]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 30
output_dir = f"models/{model_name_or_path}-epoch{train_epochs}-{timestamp}"

In [24]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                            # 输出目录
    per_device_train_batch_size=8,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=1,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=10,                                    # 模型保存步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)

In [26]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [27]:
trainer.train()

/root/miniconda3/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,5.235500
2,5.235500
3,4.909800
4,4.091900
5,2.919700
6,2.125600
7,1.405500
8,0.800600
9,0.362900
10,0.125300


/root/miniconda3/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=30, training_loss=0.9103727265067089, metrics={'train_runtime': 23.7873, 'train_samples_per_second': 5.045, 'train_steps_per_second': 1.261, 'total_flos': 357433230458880.0, 'train_loss': 0.9103727265067089, 'epoch': 30.0})

In [28]:
trainer.model.save_pretrained(output_dir)

# 测试

## 前置准备

In [29]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [30]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [31]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [32]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用微调前模型推理

In [33]:
input_text = "讲一下智慧交通观察报告-2022是谁发布的"
response, history = base_model.chat(tokenizer, query=input_text)
print(response)

很抱歉，我无法确定“智慧交通观察报告-2022”是由谁发布的，因为这个名字并不清楚。如果您能提供更多信息或者上下文，我将尽力回答您的问题。


In [34]:
response, history = base_model.chat(tokenizer, query="智慧交通观察报告的序言主题是什么？", history=history)
print(response)

《智慧交通观察报告-2022》的序言主题无法确定，因为报告的内容和主题可能因不同的编辑、出版商、研究机构或政府部门而有所不同。如果您能提供更多信息，我将尽力回答您的问题。


## 使用微调后模型推理

In [35]:
# 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp

from peft import PeftModel, PeftConfig

epochs = 30
timestamp = "20240306_160737"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"
qlora_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=4096, out_features=4608, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=4608, bias=False)

In [36]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

In [38]:
base_response, ft_response = compare_chatglm_results("讲一下智慧交通观察报告-2022是谁发布的？", base_model, qlora_model, training_tag)

问题：讲一下智慧交通观察报告-2022是谁发布的？

原始输出：
交通运输部、部属专业机构、企业联合发布的《智慧交通观察报告—2022》的事业单位为部属专业机构，而具体 releasing 为谁，目前没有具体的答案。您是指部属专业机构，还是指具体发布方？


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240306_160737）：
[gMASK]sop 讲一下智慧交通观察报告-2022是谁发布的？ 市场圆桌对话，六次行业调研，而是为行业专家建言献策，最终基于10万字手稿汇编整理。


In [39]:
base_response, ft_response = compare_chatglm_results("智慧交通观察报告的序言主题是什么", base_model, qlora_model, training_tag)

问题：智慧交通观察报告的序言主题是什么

原始输出：
交通领域涉及多个方面，包括出行、物流和交通管理。随着科技的发展，人工智能和大数据技术在交通领域中的应用越来越广泛。这些技术可以帮助提高交通系统的效率和可持续性。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240306_160737）：
[gMASK]sop 智慧交通观察报告的序言主题是什么 加快建设交通强国 智慧交通发展迎来全新机遇。


In [40]:
base_response, ft_response = compare_chatglm_results("给我讲个笑话？", base_model, qlora_model, training_tag)

问题：给我讲个笑话？

原始输出：
好的，这里有一个关于猫的笑话：

有一天，一只猫走进了一个酒吧。猫对酒保说：“喵，我想喝一杯牛奶。”

酒保惊讶地回答：“你确定你要喝牛奶？但是我们这里是酒吧啊。”

猫回答：“我知道，但是今晚我想要一个‘猫’牛奶酒。”

酒保疑惑地问：“‘猫’牛奶酒？这可是我们这里的特色菜。”

猫回答：“没错，我听说它味道很好，我想要尝试一下。”

于是，猫点了一杯“猫”牛奶酒。酒保把牛奶倒入杯中，然后加入了一些 other 的调料，比如朗姆酒、白兰地、黄酒和橙子汁。

猫尝了一口，然后皱起了眉头说：“这杯‘猫’牛奶酒’不太对劲，我想要一杯真正的牛奶。”

酒保回答：“抱歉，我这就是我们这里的特色菜，名叫‘猫’牛奶酒。”

猫想了想，说：“好吧，既然你们这么坚持，我就再尝一口。”

这次，猫喝了一口，然后说：“这杯‘猫’牛奶酒’真的很棒！我想要一杯。”

酒保高兴地说：“太好了，我们这里的‘猫’牛奶酒’就是好喝！”


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240306_160737）：
[gMASK]sop 给我讲个笑话？ | 不听老人言，男子与蛇同床| 蛇, 男子, 床, 一起, 笑话, 听老人言 | 默认 | 2023-04-07 10:08:12


In [41]:
base_response, ft_response = compare_chatglm_results("你给我说一下，那个智慧交通观察报告专栏，是哪些机构整出来的", base_model, qlora_model, training_tag)

问题：你给我说一下，那个智慧交通观察报告专栏，是哪些机构整出来的

原始输出：
很抱歉，我无法给出确切的信息，因为“智慧交通观察报告”是一个相对较新的概念，可能指的是不同机构或团队针对交通领域智能化发展的报告。这些报告可能涉及到政府、研究机构、企业等多个领域，因此，很难确定是哪些机构整出来的。如果您能提供更多具体的信息，我会尽力帮您查找相关资料。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240306_160737）：
[gMASK]sop 你给我说一下，那个智慧交通观察报告专栏，是哪些机构整出来的， 所谓的“交通领域”指的具体是指什么？ 报告专栏，指的是报告专栏 whole，不是“报告”整出来，而是指整个专栏，整个专栏指的是报告的内容，而不是结构， Whole指的是内容，而不是结构，理解了吗， Whole指的是内容，而不是结构，明白了，那么，请给我告诉我，这个专栏是通过哪些机构整出来的， 以及，这个专栏的内容，包括哪些方面，具体指的是什么？ Whole指的是内容，而不是结构，明白了，那么，请告诉我，这个专栏是通过哪些机构整出来的， 以及，这个专栏的内容，包括哪些方面，具体指的是什么？


In [42]:
base_response, ft_response = compare_chatglm_results("你给我说一下哈，智慧交通观察报告专栏是哪些机构开设的", base_model, qlora_model, training_tag)

问题：你给我说一下哈，智慧交通观察报告专栏是哪些机构开设的

原始输出：
抱歉，我无法回答您的问题，因为“智慧交通观察报告”是一个抽象的概念，它没有具体的机构开设。但是，如果您想知道关于交通领域的研究报告、数据分析和应用案例，我可以为您提供相关信息。请告诉我您想了解的方面，我会尽力为您提供帮助。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240306_160737）：
[gMASK]sop 你给我说一下哈，智慧交通观察报告专栏是哪些机构开设的? 中国交通报社联合腾讯开设《智慧交通观察报告·2022》专栏，坚持贯彻《交通强国建设纲要》《国家综合立体交通网规划纲要》等重大战略、重要规划、重点政策要求，助力推动交通运输行业智慧转型，同时，进一步探讨新技术与交通行业深度融合的典型场景和实现路径。
